## Setup and Import Libraries

In [2]:
import os
import bs4
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [3]:
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = os.getenv("HUGGINGFACE_API_KEY")

In [5]:
llm = ChatGroq(model="Gemma2-9b-It")

## Web Base Loader

In [6]:
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    )
)

In [7]:
data = loader.load()
# data

## Text Splitting

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [9]:
documents = text_splitter.split_documents(documents=data)

## Embedding and Vector Store

In [11]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [12]:
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

## Retriever

In [13]:
retriever = vector_store.as_retriever()

## Prompt Template

In [14]:
system_prompt = (
    """
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you do not know the answer, say that you do not know"
    Use three sentences maximum and keep the answer concise
    \n\n
    {context}
    """
)

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

## Chain

In [16]:
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

In [17]:
rag_chain = create_retrieval_chain(retriever=retriever, combine_docs_chain=question_answer_chain)

In [18]:
query = "What is Self Reflection"

In [19]:
response = rag_chain.invoke(
    {"input": query}
)
response

{'input': 'What is Self Reflection',
 'context': [Document(id='ccc2aa4f-8846-4c51-baea-da188df66013', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 3. Illustration of the Reflexion framework. (Image source: Shinn & Labash, 2023)\nThe heuristic function determines when the trajectory is inefficient or contains hallucination and should be stopped. Inefficient planning refers to trajectories that take too long without success. Hallucination is defined as encountering a sequence of consecutive identical actions that lead to the same observation in the environment.\nSelf-reflection is created by showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Then reflections are added into the agent’s working memory, up to three, to be used as context for querying LLM.'),
  Document(id='d54afea5-ad54-4c12-bd23-df7cda19ac69', metadata={'source': 'https://lilianweng.g

In [20]:
response['answer']

'Self-reflection is a process that allows autonomous agents to improve by examining past actions and identifying mistakes.  \n\nIt involves analyzing past trajectories and using insights to guide future decision-making, leading to more effective problem-solving.  \n\nThe Reflexion framework incorporates self-reflection by showing the agent examples of failed trajectories and ideal reflections, which are then stored in its working memory. \n'

In [21]:
query = "How do we achieve it"

response = rag_chain.invoke(
    {"input": query}
)
response['answer']

'The provided text describes several techniques for improving LLM performance on complex tasks, including task decomposition and self-reflection.  \n\nIt does not, however, explain how to achieve a specific outcome.  \n\nPlease provide more context or a specific goal for a more helpful answer. \n\n'

## Chat History

In [22]:
contextualize_system_prompt = """Given a chat history and the latest user question which might 
reference context in the chat history, formulate a standalone question which can be understood 
without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.
"""

contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [23]:
history_aware_retriever = create_history_aware_retriever(
    llm=llm, 
    retriever=retriever, 
    prompt=contextualize_prompt
)

In [24]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [25]:
question_answer_chain = create_stuff_documents_chain(
    llm=llm, 
    prompt=qa_prompt
)

In [26]:
rag_chain = create_retrieval_chain(
    retriever=history_aware_retriever, 
    combine_docs_chain=question_answer_chain)

In [27]:
chat_history = []

question1 = "What is Self-Reflection"
response1 = rag_chain.invoke({"input":question1, "chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=response1["answer"])
    ]
)

In [28]:
print(response1['answer'])

Self-reflection is a process that allows autonomous agents to learn from past experiences and improve their decision-making. 

It involves analyzing past trajectories, identifying inefficiencies or hallucinations, and using those insights to guide future actions.  This iterative process enhances the agent's ability to adapt and perform better over time.  



In [29]:
question2 = "Tell me more about it"
response2 = rag_chain.invoke({"input":question2, "chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question2),
        AIMessage(content=response2["answer"])
    ]
)

In [30]:
print(response2['answer'])

Self-reflection in LLMs involves showing the model examples of failed trajectories paired with ideal reflections that suggest improvements. These reflections are then stored in the agent's memory, acting as context for future decision-making.  

Essentially, the agent learns to identify its own mistakes and how to correct them, leading to more efficient and successful planning. 



In [31]:
chat_history

[HumanMessage(content='What is Self-Reflection', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Self-reflection is a process that allows autonomous agents to learn from past experiences and improve their decision-making. \n\nIt involves analyzing past trajectories, identifying inefficiencies or hallucinations, and using those insights to guide future actions.  This iterative process enhances the agent's ability to adapt and perform better over time.  \n", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Tell me more about it', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Self-reflection in LLMs involves showing the model examples of failed trajectories paired with ideal reflections that suggest improvements. These reflections are then stored in the agent's memory, acting as context for future decision-making.  \n\nEssentially, the agent learns to identify its own mistakes and how to correct them, leading to more efficient and suc

## Chat History with Sessions

In [32]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [33]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [34]:
config = {"configurable": {"session_id": "chat1"}}

In [35]:
response = conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config=config
)
response['answer']

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks.  \n\nThis technique, often used with LLMs, helps overcome the limitations of their finite context length and allows for more effective planning and problem-solving. \nChain of thought (CoT) and Tree of Thoughts (ToT) are examples of prompting techniques used for task decomposition. \n'

In [36]:
response = conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config=config
)
response['answer']

'There are three common ways to decompose tasks: \n\n1. Using simple prompts like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?". \n2. Employing task-specific instructions, such as "Write a story outline" for writing a novel. \n3.  Incorporating human input to guide the decomposition process. \n\n\n'

In [37]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}), AIMessage(content='Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks.  \n\nThis technique, often used with LLMs, helps overcome the limitations of their finite context length and allows for more effective planning and problem-solving. \nChain of thought (CoT) and Tree of Thoughts (ToT) are examples of prompting techniques used for task decomposition. \n', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are common ways of doing it?', additional_kwargs={}, response_metadata={}), AIMessage(content='There are three common ways to decompose tasks: \n\n1. Using simple prompts like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?". \n2. Employing task-specific instructions, such as "Write a story outline" for writing a novel. \n3.  Incorporating human inpu